In [71]:
# coding: utf-8
import requests,re
from bs4 import BeautifulSoup
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import stopwords

In [72]:
ListShazam = pd.read_csv('shazam1.csv',sep=',')
ListShazam['Paroles'] = "vide"
ListShazam['MotsFrequents'] = "vide"

In [73]:
# youtube
def importSoup(url):
    request_headers = {
       "Accept-Language": "en-US,en;q=0.5",
       "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0",
       "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
       "Referer": "http://thewgfdgebsite.com",
       "Connection": "keep-alive"
    }
    res = requests.get(url, headers = request_headers)
    if res.status_code == 200:
        html_doc =  res.text
        soup = BeautifulSoup(html_doc,"html.parser")
        return soup



In [75]:
MAX_RETRIES = 20


for i in range(len(ListShazam)):
    soup = soupParoles = all_links = Paroles = parole = urlParoles = ""
    Title, Artist = ListShazam["Title"][i], ListShazam["Artiste"][i]
    Titlep = Title.replace(' ','+')
    Artistp = Artist.replace(' ','+')
    url = r"https://search.azlyrics.com/search.php?q=" + Titlep + '+' + Artistp
    soup = importSoup(url)
    all_links = map(lambda x : x.attrs['href'] , soup.find_all("a"))
    for a in all_links:
        if re.search(r'https://www.azlyrics.com/lyrics/', a):
            urlParoles = a
            break
    if urlParoles != "":
        soupParoles = importSoup(urlParoles)
        Paroles = soupParoles.find_all("div",attrs={'class':'col-xs-12 col-lg-8 text-center'})
        parole = re.sub('\n',' ',Paroles[0].findAll('div')[6].text)
        parole = re.sub('\r',' ',parole)
        wordcount = Counter(parole.split(' '))
        wc = wordcount.most_common()
        ListShazam['Paroles'][i] = parole
        ListShazam['MotsFrequents'][i] = wc[0][0]
    else:
        ListShazam['Paroles'][i] = "vide"


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [76]:
ListShazam.head()
    

,Unnamed: 0,ID,Title,Artiste,Genre,Sortie,Duree,Note_Moyenne_itunes,#_votes_itunes,#_Shazam_fr,Paroles,MotsFrequents
0,0,402107013,Later Bitches,The Prince Karma,Dance,15 juin 2018,4,3.9,11,3016822,What the hell do you mean I've got to wait ...,you
1,1,427101360,Shallow,Lady Gaga & Bradley Cooper,Bande originale,5 oct. 2018,1,4.7,205,2166898,[Bradley Cooper:] Tell me somethin' girl Are...,the
2,2,444295408,Gam Gam,Marnik & SMACK,Dance,26 oct. 2018,3,-,-,125014,Gam gam Gam ki eleh Be be Gey tzalmavet Lo ...,Gam
3,3,448518908,Somewhere Only We Know,Reneé Dominique,-,-,-,-,-,159784,vide,vide
4,4,462933,Come And Get Your Love,Redbone,Rock,1 nov. 1973,3,-,-,2755753,Hail (hail) What's the matter with your hai...,"la,"


In [33]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hmann\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [133]:
df = ListShazam
paroles = ListShazam['Paroles']
stop_words_english = set(stopwords.words('english'))
stop_words_french = set(stopwords.words('french'))
liste =[]
def most_common_words(parole):
    p=parole.split(' ')
    p=[re.sub(r'\W+','',elmt.lower()) for elmt in p if elmt not in stop_words_english and elmt not in stop_words_french]
    wordcount = Counter(p)
    print(wordcount.most_common())
    if wordcount.most_common()[0][0] not in stop_words and wordcount.most_common()[0][0] not in 'vide':
        liste.append(wordcount.most_common())
    return wordcount.most_common()

In [134]:
ListShazam = ListShazam.dropna()
liste_words = ListShazam['Paroles'].apply(most_common_words)


[('i', 7), ('', 6), ('can', 5), ('this', 3), ('later', 3), ('bitches', 3), ('like', 3), ('another', 2), ('oh', 2), ('god', 2), ('shot', 2), ('im', 2), ('what', 1), ('hell', 1), ('mean', 1), ('ive', 1), ('got', 1), ('wait', 1), ('line', 1), ('comlpete', 1), ('total', 1), ('bullshit', 1), ('if', 1), ('show', 1), ('tits', 1), ('would', 1), ('let', 1), ('in', 1), ('just', 1), ('take', 1), ('drink', 1), ('me', 1), ('thanks', 1), ('best', 1), ('song', 1), ('ever', 1), ('love', 1), ('much', 1), ('shit', 1), ('feel', 1), ('good', 1), ('last', 1), ('horny', 1), ('right', 1), ('now', 1), ('said', 1), ('outloud', 1), ('wearing', 1), ('underwear', 1), ('so', 1), ('know', 1), ('hey', 1), ('get', 1), ('you', 1), ('one', 1), ('favor', 1), ('say', 1)]
[('shallow', 16), ('', 13), ('in', 8), ('i', 6), ('im', 4), ('times', 4), ('lady', 4), ('gaga', 4), ('were', 4), ('far', 4), ('bradley', 3), ('cooper', 3), ('tell', 2), ('somethin', 2), ('or', 2), ('need', 2), ('more', 2), ('falling', 2), ('good', 2), ('

In [135]:
liste
dictt = {}
for l in liste:
    for t in l:
        if t[0] in dictt.keys():
            dictt[t[0]] += t[1]
        else:
            dictt[t[0]] = t[1]
            
import operator
sorted_dictt = sorted(dictt.items(), key=operator.itemgetter(1), reverse=True)

In [146]:
for elem in sorted_dictt:
    if len(elem[0]) < 4:
        sorted_dictt.remove(elem)

In [148]:
sorted_dictt[:10]

[('les', 134),
 ('love', 91),
 ('plus', 84),
 ('im', 80),
 ('donne', 77),
 ('oh', 74),
 ('say', 63),
 ('get', 57),
 ('yeah', 57),
 ('man', 55)]